<a href="https://colab.research.google.com/github/Boostibot/BasicNeuralNet/blob/main/100_Final_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# The Goal


Determine if a person has diabetes based on variety of mostly binary health indicators.

Additionally, find which of the indicators are the most valuable for the purposes of classification, seek to understand the decision making process.

## Code

Split the code blocks as you need them.

### Dataset

**[Diabetes health indicators](https://www.kaggle.com/datasets/alexteboul/diabetes-health-indicators-dataset):**

The dataset is comprised of 21 mostly binary health indicators and a class healthy or diabetes. The dataset is balanced and contains 50/50 split between patients with and without diabetes. The health indicators vary greatly in distribution and importance. For example the features high blood pressure/no, age bracket, high cholesterol/no, general health indicator are highly influencial and can be readily used. On the other hand features such as eats veggies/no, has insurrance/no or even absurdly had cholesterol check in the last 5 years are all either very skewed in distribution or simply dont correspond with diabetes whatsoever.

Below are shown the distributions of each of the features. Additionally each bar is split into healty (blue) and diabetes (orange).

<img src="https://staff.utia.cas.cz/novozada/ml1/dummy.png">

To better determine which features might be useful (or if any!) I also plot the relative difference between the two classes. That is I plot the histogram $h_\text{diff}(i) = (h_d(i) - h_h(i))/h(i)$ where $i$ is the $i$-th unqiue value of the feature (histogram x value) and $h(i)$ its count in the histogram (y value). $h_d$ and $h_h$ are histograms of just the people with diabetes and without respectively.

<img src="https://staff.utia.cas.cz/novozada/ml1/dummy.png">

This largely verifies what I stated in the overview of the dataset. Nearly all features behave as one would expect, perhaps with the exception of heavy alcohol use which shows that people who often consume alcohol have significantly lower rates of diabetes than those who do. This is, however, probably caused by outliers as there is very small number of such individuals in this dataset.  

In [ ]:
import numpy as np
def load_csv_dataset(path: str, delimiter: str = ",") -> tuple:
    with open(path, "r", encoding="utf-8") as f:
        header = f.readline().rstrip("\n\r")
    labels = np.array([col.strip() for col in header.split(delimiter)], dtype=str)
    data = np.loadtxt(path, delimiter=delimiter, skiprows=1)
    if data.ndim == 1:  # single data row -> ensure 2D shape (1, n_cols)
        data = data.reshape(1, -1)
    return data, labels

from sklearn import datasets
from sklearn.model_selection import train_test_split
dataset, labels = load_csv_dataset("diabetes_binary_5050split_health_indicators_BRFSS2015.csv")

#Train test split
X, y = dataset[:,1:], dataset[:,0]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=123)

### Decision trees, random forests

I chose decision trees and random forests because they are highly interpretable. One of the goals of this project was to better understand the underlying data, which features are useful and which are not and decision making process as a whole.

Second, I wanted to experiment with random forests to see if they improve the accuracy on this noisy dataset.

For both models I used the implementation provided by scikit-learn. The used formulation corresponds directly to the one introduced in these labs.

In [ ]:
class MyDummyClassifier:
  def __init__(self):
    pass

  def fit(self, X, y):
    n_samples, n_features = X.shape
    self._classes = np.unique(y)
    self._priors = [X[c==y].shape[0] / float(n_samples) for c in self._classes]

  def predict(self, X):
    print(self._priors)
    return np.random.choice(self._classes, X.shape[0], p=self._priors)

### Train your classifier(s) and predict the testing data

In [ ]:
def accuracy(y_true, y_pred):
  accuracy = np.sum(y_true == y_pred) / len(y_true)
  return accuracy

# Design a model
clf = MyDummyClassifier()

# 4. Train a model
clf.fit(X_train, y_train)

# 5. Make predictions
predictions = clf.predict(X_test)

# 6. Evaluate predictions
print("Dummy classifier has accuracy", accuracy(y_test, predictions))

[0.12625, 0.87375]
Dummy classifier has accuracy 0.75


## Discuss the results

We find that surprisingly even very small trees predict the data very well. We also find that large trees are prone to overfitting and achive worse results than even the smallest tree tested.

Random forests ever-so-slightly improve on the test results but are probably not worth doing for their additional cost and train time for this dataset.

What do the trees look like? Heres a tree with 20 leaf nodes visualized with grapviz.

...

as you can see some of the nodes are redunant and the tree can be simplified. For this we write this rather complicated recursive function

In [ ]:
def collapse_same_class_subtrees(clf):
    """
    Modify clf.tree_ in place to collapse any node whose entire subtree
    predicts a single class into a leaf node.
    """
    from sklearn.tree import _tree
    tree = clf.tree_
    children_left = tree.children_left
    children_right = tree.children_right
    value = tree.value

    def recurse(node):
        # if node is leaf => return predicted class index
        if children_left[node] == _tree.TREE_LEAF and children_right[node] == _tree.TREE_LEAF:
            return np.argmax(value[node])

        # otherwise check children
        left_child = children_left[node]
        right_child = children_right[node]
        left_class = recurse(left_child)
        right_class = recurse(right_child)

        # if both children subtrees exist and both predict same single class
        if (left_class is not None) and (right_class is not None) and (left_class == right_class):
            # collapse node -> make it a leaf
            tree.children_left[node] = _tree.TREE_LEAF
            tree.children_right[node] = _tree.TREE_LEAF
            tree.feature[node] = _tree.TREE_UNDEFINED
            tree.threshold[node] = -2.0
            # aggregate counts/values from children (their value equals subtree counts)
            tree.value[node] = tree.value[left_child] + tree.value[right_child]
            return left_class
        else:
            return None

    recurse(0)  # root

Applying this function and regenerating the graph shows us this rather simple decision tree.

...

Lastly, we can see that most paths through the tree terminate rather quickly in one of the leafs, yet there are some longer paths possible as well. Indeed, if we look at the counts we can see that just the top few nodes capture most of the instances. So what would it look like if we simplified further?

Here is (simplified) tree of just two levels. The classification using this tree is extemely simple, yet even such a tiny tree gives better results than the overly large tree we started with!

...